# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [1]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date
from datetime import timedelta
import json
import os
from dotenv import load_dotenv

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [2]:
today = datetime.now()
start_point = today - timedelta(days=7)

today = round(today.timestamp())
start_point = round(start_point.timestamp())
print(today)
print(start_point)

1753894455
1753289655


In [65]:
# Configuration API
API_KEY = os.getenv("API_KEY")
BASE_URL = "http://api.openweathermap.org/data/2.5"

# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Rome, Lazio", "Amsterdam", "Vienna"]

In [4]:
### 🗺️ Obtenir les coordonnées
def get_city_coordinates(city):
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """
    # load APi key from dotenv 
    load_dotenv()
    # url 
    URL ="http://api.openweathermap.org/geo/1.0/direct"
    # our parameters that we will use to make the rquest
    params = {
        "q": c,
        "appid" : API_KEY,
        "limit" : 1
    }
    #send the request and get back json object the extract lat and lon 
    try : 
        responde = requests.get(URL, params=params)
        geo = responde.json()
        lat = geo[0]['lat']
        lon = geo[0]['lon']
        return lat, lon
    # errors 
    except requests.exceptions.HTTPError as http_err :
        if responde.status_code == 404 :
            print(f"No coordinates found for {city}")
        else:
            print(f"Failed to get coordinates {city}, {http_err}")
            return None, None
    except Exception as err:
        print(f"UNexpected Error {err}")
        return None, None
        
for c in CITIES :
    lat, lon = get_city_coordinates(c)
    print(f"{c} {get_city_coordinates(c)}")
    

Paris (48.8588897, 2.3200410217200766)
Berlin (52.5170365, 13.3888599)
Madrid (40.4167047, -3.7035825)
Rome (41.8933203, 12.4829321)
Amsterdam (52.3727598, 4.8936041)
Vienna (48.2083537, 16.3725042)


In [5]:
### 💡 Première requête guidée

def get_weather_data(city):
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    load_dotenv()
    api_key = os.getenv("API_KEY")
    # URL : current weather data
    url = f"{BASE_URL}/weather"
    # Paramètres à compléter
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # Celsius
        'lang': 'en'
    }
    try :
        responde = requests.get(url, params=params)
        responde.raise_for_status()
        data = responde.json()
        return data
    except requests.exceptions.HTTPError as http_err:
        if responde.status_code == 404 :
            print(f"City is not found {city}")
        else :
            print(f"HTTP Error Is Occurred :{http_err}")
        return None
    except Exception as err:
        print(f"Unexpected Error {err}")
        return None
get_weather_data("Paris")


{'coord': {'lon': 2.3488, 'lat': 48.8534},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 24.18,
  'feels_like': 24.22,
  'temp_min': 22.9,
  'temp_max': 25.04,
  'pressure': 1018,
  'humidity': 60,
  'sea_level': 1018,
  'grnd_level': 1008},
 'visibility': 10000,
 'wind': {'speed': 4.12, 'deg': 10},
 'clouds': {'all': 75},
 'dt': 1753894367,
 'sys': {'type': 1,
  'id': 6550,
  'country': 'FR',
  'sunrise': 1753849276,
  'sunset': 1753903957},
 'timezone': 7200,
 'id': 2988507,
 'name': 'Paris',
 'cod': 200}

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [57]:
# Configuration Visual Crossing
load_dotenv()
VC_API_KEY = os.getenv("API_KEY2")
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

    url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"
    params = {
        'key': VC_API_KEY,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
    }
    try :
        response = requests.get(url, params=params)
        response.raise_for_status()
        histo_weather = response.json()
        data_historic = histo_weather["days"]
        print(data_historic)
        return data_historic
    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404 :
            print(f"city is not found {city}, or wronge date{start_date}, {end_date}")
        else :
            print(f"HTTP Error Is Occurred: {http_err}")
    except Exception as err:
        print(f"Unexpected Error {err}")
# weather_h = get_historical_weather("Paris", start_point, today)
# print(weather_h)


In [64]:
cities_hw = []
for c in CITIES :
    data_historic = get_historical_weather(c, start_point, today)
    print(data_historic)        
    # data_historic = get_historical_weather(c, start_point, today)
    # print(data_historic)
#     for day in data_historic :
#         cities_hw.append({
#             "city" : c,
#             'datetime': day["datetime"],
#             'temperature': day["temp"],
#             'humidity': day["humidity"],
#             'precipitation': day["precip"],
#             'wind_speed': day["windspeed"],
#             'pressure': day["pressure"]
#         })
# print(cities_hw)

HTTP Error Is Occurred: 429 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Paris/1753289655/1753894455?key=CG7WRZ9SZTKEK4TBHZ5245BSC&include=days&elements=datetime%2Ctemp%2Chumidity%2Cprecip%2Cwindspeed%2Cpressure
None
HTTP Error Is Occurred: 429 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Berlin/1753289655/1753894455?key=CG7WRZ9SZTKEK4TBHZ5245BSC&include=days&elements=datetime%2Ctemp%2Chumidity%2Cprecip%2Cwindspeed%2Cpressure
None
HTTP Error Is Occurred: 429 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Madrid/1753289655/1753894455?key=CG7WRZ9SZTKEK4TBHZ5245BSC&include=days&elements=datetime%2Ctemp%2Chumidity%2Cprecip%2Cwindspeed%2Cpressure
None
HTTP Error Is Occurred: 429 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Rome,%20Lazio/17532896

### 🎯 Mission pratique
Récupérez les données des 30 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [109]:
# Charger les variables d’environnement
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Fonction pour obtenir les coordonnées d’une ville
def get_city_coordinates(city):
    url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": city,
        "limit": 1,
        "appid": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0]["lat"], data[0]["lon"]
        else:
            print(f"[!] No coordinates found for {city}")
            return None, None
    except Exception as e:
        print(f"[ERROR] Failed to get coordinates for {city}: {e}")
        return None, None

# Fonction pour obtenir la qualité de l’air
def get_air_quality(lat, lon):
    url = "http://api.openweathermap.org/data/2.5/air_pollution"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        air_q = response.json()
        if air_q and "list" in air_q and air_q["list"]:
            info = air_q['list'][0]
            aqi = info.get('main', {})
            comp = info.get('components', {})
            return aqi, comp
        else:
            print(f"[!] No air quality data returned for {lat}, {lon}")
            return None, None
    except Exception as e:
        print(f"[ERROR] Failed to get air quality data: {e}")
        return None, None

# Liste des villes
CITIES = ['Paris', 'London', 'Berlin']

# Collecte des données
all_air_q = []
for city in CITIES:
    lat, lon = get_city_coordinates(city)
    if lat is None or lon is None:
        continue  # skip this city if coordinates not found

    aqi, comp = get_air_quality(lat, lon)
    if aqi is None or comp is None:
        continue  # skip if API failed

    all_air_q.append({
        "city": city,
        "lat": lat,
        "lon": lon,
        "aqi": aqi.get("aqi"),
        "co": comp.get("co"),
        "no": comp.get("no"),
        "no2": comp.get("no2"),
        "o3": comp.get("o3"),
        "so2": comp.get("so2"),
        "pm2_5": comp.get("pm2_5"),
        "pm10": comp.get("pm10"),
        "nh3": comp.get("nh3")
    })

# Affichage des résultats
print(all_air_q)

[{'city': 'Paris', 'lat': 48.8588897, 'lon': 2.3200410217200766, 'aqi': 2, 'co': 108.68, 'no': 0.01, 'no2': 1.52, 'o3': 68.06, 'so2': 0.31, 'pm2_5': 2.44, 'pm10': 2.65, 'nh3': 1.92}, {'city': 'London', 'lat': 51.5073219, 'lon': -0.1276474, 'aqi': 2, 'co': 119.01, 'no': 0.2, 'no2': 9.35, 'o3': 70.34, 'so2': 4.55, 'pm2_5': 4.13, 'pm10': 6.67, 'nh3': 0.67}, {'city': 'Berlin', 'lat': 52.5170365, 'lon': 13.3888599, 'aqi': 2, 'co': 124.42, 'no': 0.03, 'no2': 7.2, 'o3': 62.21, 'so2': 1.03, 'pm2_5': 1.94, 'pm10': 2.55, 'nh3': 7.36}]


In [110]:
df_weather = pd.DataFrame(all_air_q)

In [111]:
df_weather

,city,lat,lon,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Paris,48.858890,2.320041,2,108.68,0.01,1.52,68.06,0.31,2.44,2.65,1.92
1,London,51.507322,-0.127647,2,119.01,0.20,9.35,70.34,4.55,4.13,6.67,0.67
2,Berlin,52.517037,13.388860,2,124.42,0.03,7.20,62.21,1.03,1.94,2.55,7.36



## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [8]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [9]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [10]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites